## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

导入HMM参数，初始化所需的起始概率矩阵，转移概率矩阵，发射概率矩阵

In [19]:
import pickle
import numpy as np

In [20]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的ord作为行key
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

定义待处理的句子

In [21]:
# TODO: 将input_sentence中的xxx替换为你的姓名（1分）
input_sentence = "刘乔安是一名优秀的学生"


实现viterbi算法，并以此进行中文分词

<font color="red">阐释</font>

Viterbi算法利用了动态规划的思想，重复利用信息，从而降低时间复杂度。
- 首先是初始化部分，将第1个位置的初始值 $P(h_1)$ 设为 $a_{0j}b_j(v_1)$。
- 然后采用动态规划对其余位置的最大概率进行赋值。
- 最后，根据`path`记录的路径，回溯得到每个位置最有可能的隐状态。

In [22]:
def viterbi(sent_orig, start_prob, trans_mat, emission_mat):
    """
    viterbi算法进行中文分词

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]
    
    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float) # shape(2,len(sent_ord))
    
    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)
    
    #  TODO: 第一个位置的最大概率值计算（1分）
    dp[:,0] = start_prob * emission_mat[:,sent_ord[0]]
    
    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）（2分）
    for i in range(1,len(sent_ord)):
        array_for_B = dp[:,i-1] * trans_mat[:,0] * emission_mat[:,sent_ord[i]]
        array_for_I = dp[:,i-1] * trans_mat[:,1] * emission_mat[:,sent_ord[i]]

        index_for_B = np.argmax(array_for_B)
        index_for_I = np.argmax(array_for_I)

        dp[0,i] = array_for_B[index_for_B]
        dp[1,i] = array_for_I[index_for_I] 
        path[0][i] = index_for_B
        path[1][i] = index_for_I
    
    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]
    
    #  TODO：计算labels每个位置上的值（填充labels矩阵）（1分）
    labels[len(sent_ord)-1] = np.argmax(dp[:,len(sent_ord)-1])
    for i in range(len(sent_ord)-2,0,-1):
        labels[i] = path[labels[i+1]][i]
    
    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str
            

In [23]:
print("viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

viterbi算法分词结果： 刘乔安是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值

<font color="red">阐释</font>

- 首先，将 $\alpha_1(j)$ 初始化为 $a_{0j}b_j(v_1), 1\le j \le N$。
- 再利用类似于Viterbi算法的动态规划思想，计算其余位置的概率 $\alpha_t(j) = \sum_{i=1}^N \alpha_{t-1}(i)a_{ij}b_j(v_t), N=2$。（注意此处对前一位置的所有情况进行求和，而Viterbi算法是求最大值。）
- 最后使用公式计算整个句子背后的隐状态概率 $P(v | h) = \sum_{i=1}^N \alpha_T(i) a_{iF}$。

In [24]:
def compute_prob_by_forward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    前向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算（1分）
    dp[:,0] = start_prob * emission_mat[:,sent_ord[0]]
    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in range(1,len(sent_ord)):
        dp[0,i] = np.sum(dp[:,i-1] * trans_mat[:,0] * emission_mat[0,sent_ord[i]])
        dp[1,i] = np.sum(dp[:,i-1] * trans_mat[:,1] * emission_mat[1,sent_ord[i]])
      
    # pass

    return sum([dp[i][len(sent_ord)-1] for i in range(2)])

实现后向算法，计算该句子的概率值

<font color="red">阐释</font>

- 首先，将终末位置的概率初始化为1，即 $\beta_T(i) = a_{iF} = 1, 1 \le i \le N, N=2$。
- 再使用动态规划计算其余位置的概率， 即 $\beta_t(i) = \sum_{j=1}^N a_{ij}b_j(v_{t+1})\beta_{t+1}(j), 1\le i \le N, 1 \le t < T$。
- 最后计算整个句子背后的隐状态概率 $P(v | h) = \beta_1(0) = \sum_{j=1}^N a_{0j}b_j(v_1) \beta_1(j)$。

In [25]:
def compute_prob_by_backward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    后向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化（1分）
    dp[:,len(sent_ord)-1] = 1 
    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in range(len(sent_ord)-2,-1,-1):
        # print(trans_mat[0,:].shape)
        dp[0,i] = np.sum(dp[:,i+1] * trans_mat[0,:] * emission_mat[:,sent_ord[i+1]])
        dp[1,i] = np.sum(dp[:,i+1] * trans_mat[1,:] * emission_mat[:,sent_ord[i+1]])

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [26]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 1.050042528069508e-33
后向算法概率： 1.0500425280695082e-33


## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有7处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [27]:
import re
import functools

In [28]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")

def white_space_tokenize(corpus):
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到 [["i", "am", "happy"], ["i", "have", "N", "apples"]]

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(
        filter(lambda tkn: len(tkn)>0, _splitor_pattern.split(_digit_pattern.sub("N", stc.lower())))) for stc in corpus
    ]
    
    return tokeneds

编写相应函数构建BPE算法需要用到的初始状态词典

<font color="red">阐释</font>

本题只需对句子中每一个词按照字符进行切分，添加空格。最后在词的末尾加上`</w>`。

In [29]:
def build_bpe_vocab(corpus):
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
     }

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()
    
    # TODO: 完成函数体（1分）
    for sentence in tokenized_corpus:
        for word in sentence:
            key = " ".join(word)+" </w>"
            if key in bpe_vocab:
                bpe_vocab[key] += 1
            else:
                bpe_vocab[key] = 1

    return bpe_vocab

编写所需的其他函数

<font color="red">阐释</font>

第一个函数`get_bigram_freq`从第一步清洗后的数据得到每个bigram的频数。这个函数的实现过程也较为简单，只需要对每一词进行更加精细的划分即可。

In [30]:
def get_bigram_freq(bpe_vocab):
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：
    输入 bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple(str, str), int] - "bigram->频数"的词典
    """

    bigram_freq = dict()
    
    # TODO: 完成函数体（1分）
    for word, freq in bpe_vocab.items():
        characters = word.split(" ")
        for i in range(len(characters)-1):
            key = (characters[i], characters[i+1])
            if key in bigram_freq:
                bigram_freq[key] += freq
            else:
                bigram_freq[key] = freq

    return bigram_freq

<font color="red">阐释</font>

第二个函数`efresh_bpe_vocab_by_merging_bigram`在“单词分词状态->频数”的字典中，将每个key中存在bigram的位置合并。
为了方便，我先定义了一个`mask`用于探测字典的每一个key中有无指定的bigram，再把合并后的新字典返回。

In [31]:
def refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab):
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：
    输入 bigram=('i', '</w>')，old_bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }

    Args:
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """
    
    new_bpe_vocab = dict()

    # TODO: 完成函数体（1分）
    mask = f"{bigram[0]} {bigram[1]}"
    new_mask = bigram[0]+bigram[1]
    for word, freq in old_bpe_vocab.items():
        if mask in word:
            key = word.split(mask)
            key.insert(1,new_mask)
            key = "".join(key)
            new_bpe_vocab[key] = freq
        else:
            new_bpe_vocab[word] = freq
    return new_bpe_vocab

<font color="red">阐释</font>

第三个函数`get_bpe_tokens`根据“单词分词状态->频数”字典，返回每个BPE分词以及它出现的频数，并且以BPE分词长度递减的顺序返回一个数组。我首先用一个字典统计每种BPE分词的频数，再用`sorted`函数对分词长度排序后返回。注意此处`"</w>"`算作长度为1的分词。

In [32]:
def get_bpe_tokens(bpe_vocab):
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：
    输入 bpe_vocab=
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    得到
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
     ]

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 完成函数体（2分）
    def length_(string):
        if '</w>' not in string:
            return len(string)
        else:
            return len(string)-3
    
    bpe_tokens = dict()
    for word, freq in bpe_vocab.items():
        word_list = word.split(' ')
        for key in word_list:
            if key in bpe_tokens:
                bpe_tokens[key] += freq
            else:
                bpe_tokens[key] = freq
    bpe_tokens = sorted(map(lambda itm: (itm[0], itm[1]), bpe_tokens.items()),key = lambda itm: length_(itm[0]), reverse=True)
    return bpe_tokens

<font color="red">阐释</font>

第四个函数`print_bpe_tokenize`按照长度降序对单词进行BPE分词。这里采用递归函数来完成这个过程。
- 首先确定终止情况，即`sub_word`是一个空字符串时，直接返回空字符串。
- 利用`bpe_tokens`按照分词长度递减排序的特性，遍历这个数组，找到第一个匹配的分词后，按照这个分词截断整个`sub_word`，再对左右部分递归调用。
- 若没有找到任何一个匹配的分词，则返回`"<unknown>"`

In [33]:
def print_bpe_tokenize(word, bpe_tokens):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，
    或者剩余部分无法匹配（该部分整体由"<unknown>"代替）
    
    例1：
    输入 word="supermarket", bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]
    最终打印 "su per <unknown>"

    例2：
    输入 word="shanghai", bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
    最终打印 "sh an g hai </w>"

    Args:
        word: str - 待分词的单词str
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 请尝试使用递归函数定义该分词过程（2分）
    def bpe_tokenize(sub_word):
        if sub_word == "": return ""
        for gram, freq in bpe_tokens:
            if gram in sub_word:
                subword_list = sub_word.split(gram)
                left_gram = bpe_tokenize(subword_list[0])
                right_gram = bpe_tokenize(subword_list[1])
                if left_gram != '': 
                    left_gram += ' '
                if right_gram != '':
                    right_gram = ' ' + right_gram
                return left_gram + gram + right_gram
        return "<unknown>"
        pass

    res = bpe_tokenize(word+"</w>")
    print(res)

开始读取数据集并训练BPE分词器

In [34]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


<font color="red">阐释</font>

在循环体内，
- 首先调用`get_bigram_freq`函数，得到每个bigram出现的频次。
- 找到具有最大频次的`bigram`，作为接下来要合并的候选者。
- 调用`refresh_bpe_vocab_by_merging_bigram`函数，得到更新后的`training_bpe_vocab`。

In [35]:
training_iter_num = 300

training_bpe_vocab = build_bpe_vocab(training_corpus)
for i in range(training_iter_num):
    # TODO: 完成训练循环内的代码逻辑（2分）
    bigram_freq = get_bigram_freq(training_bpe_vocab)
    bigram = max(bigram_freq, key = lambda x: bigram_freq[x])
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(bigram,training_bpe_vocab)

training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)

测试BPE分词器的分词效果

In [36]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n atur al lan gu age pro ce s sing</w>
